In [ ]:
!pip install pyspark==3.5.1

In [ ]:
import os
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
spark = SparkSession.builder.appName("Linear-Regression-California-Housing").getOrCreate()

In [ ]:
spark

In [ ]:
HOUSING_DATA = 'cal_housing.data'

In [ ]:
schema = StructType([
    StructField("long", FloatType(), nullable=True),
    StructField("lat", FloatType(), nullable=True),
    StructField("medage", FloatType(), nullable=True),
    StructField("totrooms", FloatType(), nullable=True),
    StructField("totbdrms", FloatType(), nullable=True),
    StructField("pop", FloatType(), nullable=True),
    StructField("houshlds", FloatType(), nullable=True),
    StructField("medinc", FloatType(), nullable=True),
    StructField("medhv", FloatType(), nullable=True)]
)

In [ ]:
housing_df = spark.read.csv(path=HOUSING_DATA, schema=schema).cache()

In [ ]:
housing_df.take(5)

In [ ]:
housing_df.show(5)

In [ ]:
housing_df.columns

In [ ]:
housing_df.printSchema()

In [ ]:
housing_df.select('pop','totbdrms').show(10)

In [ ]:
result_df = housing_df.groupBy("medage").count().sort("medage", ascending=False)

In [ ]:
result_df.show(10)

In [ ]:
result_df.toPandas().plot.bar(x='medage',figsize=(14, 6))

In [ ]:
(housing_df.describe().select(
                    "summary",
                    F.round("medage", 4).alias("medage"),
                    F.round("totrooms", 4).alias("totrooms"),
                    F.round("totbdrms", 4).alias("totbdrms"),
                    F.round("pop", 4).alias("pop"),
                    F.round("houshlds", 4).alias("houshlds"),
                    F.round("medinc", 4).alias("medinc"),
                    F.round("medhv", 4).alias("medhv"))
                    .show())

In [ ]:
housing_df = housing_df.withColumn("medhv", col("medhv")/100000)

In [ ]:
housing_df.show(2)

In [ ]:
housing_df.columns

In [ ]:
housing_df = (housing_df.withColumn("rmsperhh", F.round(col("totrooms")/col("houshlds"), 2))
                       .withColumn("popperhh", F.round(col("pop")/col("houshlds"), 2))
                       .withColumn("bdrmsperrm", F.round(col("totbdrms")/col("totrooms"), 2)))

In [ ]:
housing_df.show(5)

In [ ]:
# Re-order and select columns
housing_df = housing_df.select("medhv", 
                              "totbdrms", 
                              "pop", 
                              "houshlds", 
                              "medinc", 
                              "rmsperhh", 
                              "popperhh", 
                              "bdrmsperrm")

In [ ]:
featureCols = ["totbdrms", "pop", "houshlds", "medinc", "rmsperhh", "popperhh", "bdrmsperrm"]

In [ ]:
assembler = VectorAssembler(inputCols=featureCols, outputCol="features") 

In [ ]:
assembled_df = assembler.transform(housing_df)

In [ ]:
assembled_df.show(10, truncate=False)